In [1]:
from huggingface_hub import login, InferenceClient
from transformers import AutoTokenizer

In [2]:
import os
import sys
import json
from dotenv import load_dotenv
from openai import OpenAI
import google.generativeai
from anthropic import Anthropic
import gradio as gr
import subprocess
from IPython.display import Markdown, display, update_display
from huggingface_hub import login

In [3]:
HF_TOKEN = os.getenv("HF_TOKEN")
login(HF_TOKEN, add_to_git_credential=True)

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [5]:
code_qwen = "Qwen/CodeQwen1.5-7B-Chat"
code_qwen_endpoint = "https://abh36oub208wpbqp.us-east4.gcp.endpoints.huggingface.cloud"

In [6]:
## System prompt
system_prompt ="You are a very helpful assistant that can reimplement python codes into high performing C++ for google colab"
system_prompt+="You are to provide only C++ code. use comments appropriately and do not provide any explanation other than comments."
system_prompt+="The C++ version needs to reproduce the exact same output that the python code does. the execution time for the C++ should be the fastest"

def get_user_prompt(python_code):
  user_prompt="rewrite the python code in C++ with the fastest possible execution. the C++ should have identical outcomes like the python in the least time."
  user_prompt+="respond only in C++ code. dont explain any bit other than the comments."
  user_prompt+="pay attention to the data types. Make sure there is no int overflow. Also make sure to include all the necessary libraries for C++ like iomanip."
  user_prompt+=f"The following one is the python code that you need to convert into C++: {python_code}"
  return user_prompt

def get_messages(python_code):
  return [
      {"role": "system", "content": system_prompt},
      {"role": "user", "content": get_user_prompt(python_code)}
  ]

In [7]:
pi = """
import time

def calculate(iterations, param1, param2):
    result = 1.0
    for i in range(1, iterations+1):
        j = i * param1 - param2
        result -= (1/j)
        j = i * param1 + param2
        result += (1/j)
    return result

start_time = time.time()
result = calculate(100_000_000, 4, 1) * 4
end_time = time.time()

print(f"Result: {result:.12f}")
print(f"Execution Time: {(end_time - start_time):.6f} seconds")
"""

In [24]:
python_hard = """# Be careful to support large number sizes

def lcg(seed, a=1664525, c=1013904223, m=2**32):
    value = seed
    while True:
        value = (a * value + c) % m
        yield value

def max_subarray_sum(n, seed, min_val, max_val):
    lcg_gen = lcg(seed)
    random_numbers = [next(lcg_gen) % (max_val - min_val + 1) + min_val for _ in range(n)]
    max_sum = float('-inf')
    for i in range(n):
        current_sum = 0
        for j in range(i, n):
            current_sum += random_numbers[j]
            if current_sum > max_sum:
                max_sum = current_sum
    return max_sum

def total_max_subarray_sum(n, initial_seed, min_val, max_val):
    total_sum = 0
    lcg_gen = lcg(initial_seed)
    for _ in range(20):
        seed = next(lcg_gen)
        total_sum += max_subarray_sum(n, seed, min_val, max_val)
    return total_sum

# Parameters
n = 10000         # Number of random numbers
initial_seed = 42 # Initial seed for the LCG
min_val = -10     # Minimum value of random numbers
max_val = 10      # Maximum value of random numbers

# Timing the function
import time
start_time = time.time()
result = total_max_subarray_sum(n, initial_seed, min_val, max_val)
end_time = time.time()

print("Total Maximum Subarray Sum (20 runs):", result)
print("Execution Time: {:.6f} seconds".format(end_time - start_time))
"""

In [8]:
tokenizer = AutoTokenizer.from_pretrained(code_qwen)
messages = get_messages(pi)
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)

tokenizer_config.json:   0%|          | 0.00/972 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/1.42M [00:00<?, ?B/s]

In [10]:
print(messages)

[{'role': 'system', 'content': 'You are a very helpful assistant that can reimplement python codes into high performing C++ for google colabYou are to provide only C++ code. use comments appropriately and do not provide any explanation other than comments.The C++ version needs to reproduce the exact same output that the python code does. the execution time for the C++ should be the fastest'}, {'role': 'user', 'content': 'rewrite the python code in C++ with the fastest possible execution. the C++ should have identical outcomes like the python in the least time.respond only in C++ code. dont explain any bit other than the comments.pay attention to the data types. Make sure there is no int overflow. Also make sure to include all the necessary libraries for C++ like iomanip.The following one is the python code that you need to convert into C++: \nimport time\n\ndef calculate(iterations, param1, param2):\n    result = 1.0\n    for i in range(1, iterations+1):\n        j = i * param1 - param

In [11]:
print(text)

<|im_start|>system
You are a very helpful assistant that can reimplement python codes into high performing C++ for google colabYou are to provide only C++ code. use comments appropriately and do not provide any explanation other than comments.The C++ version needs to reproduce the exact same output that the python code does. the execution time for the C++ should be the fastest<|im_end|>
<|im_start|>user
rewrite the python code in C++ with the fastest possible execution. the C++ should have identical outcomes like the python in the least time.respond only in C++ code. dont explain any bit other than the comments.pay attention to the data types. Make sure there is no int overflow. Also make sure to include all the necessary libraries for C++ like iomanip.The following one is the python code that you need to convert into C++: 
import time

def calculate(iterations, param1, param2):
    result = 1.0
    for i in range(1, iterations+1):
        j = i * param1 - param2
        result -= (1/j

In [9]:
client = InferenceClient(
    code_qwen_endpoint,
    token=HF_TOKEN
)

stream = client.text_generation(
    text,
    stream=True,
    details=True,
    max_new_tokens=3000
)

for r in stream:
    print(r.token.text, end="")

Here is the C++ version of the Python code:

```cpp
#include <iostream>
#include <iomanip>
#include <chrono>

double calculate(int iterations, int param1, int param2) {
    double result = 1.0;
    for (int i = 1; i <= iterations; ++i) {
        int j = i * param1 - param2;
        result -= 1.0 / j;
        j = i * param1 + param2;
        result += 1.0 / j;
    }
    return result;
}

int main() {
    auto start_time = std::chrono::high_resolution_clock::now();
    double result = calculate(100000000, 4, 1) * 4;
    auto end_time = std::chrono::high_resolution_clock::now();

    std::cout << "Result: " << std::fixed << std::setprecision(12) << result << std::endl;
    std::cout << "Execution Time: " << std::chrono::duration<double>(end_time - start_time).count() << " seconds" << std::endl;

    return 0;
}
```

This C++ code does the same thing as the Python code. It calculates the result of the function `calculate` and prints it out along with the execution time. The `calculate` fun

In [13]:
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
CLAUDE_API_KEY = os.getenv("ANTHROPIC_API_KEY")

In [14]:
openai_client = OpenAI(api_key=OPENAI_API_KEY)
claude_client = Anthropic(api_key=CLAUDE_API_KEY)

In [16]:
OPENAI_MODEL = 'gpt-4o'
CLAUDE_MODEL = 'claude-sonnet-4-0'

In [18]:
def stream_gpt(python_code):
  stream = openai_client.chat.completions.create(
      model = OPENAI_MODEL,
      messages = get_messages(python_code),
      stream = True
  )
  reply =""
  for chunk in stream:
    piece = chunk.choices[0].delta.content or ""
    reply+=piece
    yield reply.replace("```cpp\n","").replace("```", "")


def stream_claude(python_code):
  reply = claude_client.messages.stream(
      model = CLAUDE_MODEL,
      max_tokens=2000,
      system=system_prompt,
      messages=[{"role": "user", "content": get_user_prompt(python_code)}]
  )
  response =""
  with reply as stream:
    for text in stream.text_stream:
      response+=text
      yield response.replace("```cpp\n","").replace("```", "")

In [19]:
def stream_qwen(python_code):
    tokeninzer = AutoTokenizer.from_pretrained(code_qwen)
    messages = get_messages(python_code)
    text = tokeninzer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
    )
    client = InferenceClient(
        code_qwen_endpoint,
        token=HF_TOKEN
    )
    stream = client.text_generation(
        text,
        stream=True,
        details=True,
        max_new_tokens=3000
    )
    result = ""
    for r in stream:
        result+=r.token.text or ""
        yield result

In [20]:
def optimize(python_code, model):
    if model == "GPT":
        result = stream_gpt(python_code)
    elif model == "CLAUDE":
        result = stream_claude(python_code)
    elif model == "QWEN":
        result = stream_qwen(python_code)
    else:
        raise ValueError("Unknown model")
    
    for r in result:
        yield r

In [21]:
def execute_python(python_code):
  try:
    result = subprocess.run(['python', '-c', python_code], capture_output=True, text=True, check=True)
    return result.stdout
  except subprocess.CalledProcessError as e:
    return f"Error:\n{e.stderr}"

In [22]:
def execute_cpp(cpp_code):
  try:
    with open("temp.cpp", "w") as f:
      f.write(cpp_code)

    compile_result = subprocess.run(['g++', "temp.cpp", '-o', 'temp_cpp_executable'], capture_output=True, text=True, check=True)
    if compile_result.stderr:
      return f"Compilation Errors:\n{compile_result.stderr}"

    run_result = subprocess.run(['./temp_cpp_executable'], capture_output=True, text=True, check=True)
    return run_result.stdout
  except subprocess.CalledProcessError as e:
    return f"Execution Error:\n{e.stderr}"
  except FileNotFoundError:
    return "Error: g++ compiler not found. Please ensure it is installed."

In [25]:
with gr.Blocks() as ui:
    gr.Markdown("## Convert Code from python to c++")
    with gr.Row():
        python = gr.Textbox(label="Python Code:", value=python_hard, lines=15)
        cpp = gr.Textbox(label="C++ code", lines=15)
    
    with gr.Row():
        model = gr.Dropdown(["GPT", "CLAUDE", "QWEN"], label="Select a Model", value="GPT")
    
    with gr.Row():
        convert = gr.Button("Convert Code")
    
    with gr.Row():
        python_run = gr.Button("Run Python")
        cpp_run = gr.Button("Run C++")
    with gr.Row():
        python_out = gr.TextArea(label="Python Result: ", elem_classes=["python"])
        cpp_out = gr.TextArea(label="C++ Result: ", elem_classes=["cpp"])
    
    convert.click(optimize, inputs=[python,model], outputs=[cpp])
    python_run.click(execute_python, inputs=[python], outputs=[python_out])
    cpp_run.click(execute_cpp, inputs=[cpp], outputs=[cpp_out])

ui.launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.
